### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
import pandas as pd

player_purchase = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(player_purchase)

## Player Count

* Display the total number of players


In [44]:
# To get total players I checked for unique SN (I assume that means screen name), by creating a list of unique players
# and then applying .nunique to get the player count. I saw the homework returned this as a dataframe series, but I used the 
# number below in calculating percentages so it was easier to keep the value as an integer and just print a formatted string return. 

unique_players = purchase_data["SN"].nunique()
print(f"Total Number of Players: {unique_players}")

Total Number of Players: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [45]:
# The first four lines calculate the values and then the simplest way to display them into a table was just to create a new dataframe
# I probably could have just transformed the dataframe and rename the columns as well, but because it was only 4 columns this was a quick
# way to do it. In the following sections I use groupby and aggregate to show another way to do this that is simpler.

unique_items = len(purchase_data["Item ID"].unique())
average_purchase = round(purchase_data["Price"].mean(), 2)
total_purchases = len(purchase_data)
total_revenue = round(purchase_data["Price"].sum(), 2)

# Create a new data frame by creating a dictionary of the values and converting to dataframe. 

purchase_analysis = {"Number of Unique Items": unique_items, "Average Price": average_purchase,
                     "Number of Purchases": total_purchases, "Total Revenue": total_revenue}
purchasing_total = pd.DataFrame(purchase_analysis, columns = ['Number of Unique Items',
                                                             'Average Price', 'Number of Purchases', 'Total Revenue'], index=[0])

# This just adds the formatting of the dollars. I like keeping my dataframe unformatted because it is easier to work with the raw numbers
# and then add the formatting when needed. Also, I'm pretty sure an apply or lambda could make these two lines into one. Or doing it in a
# table styles property could also work. It took a while for me to get the hang of css selectors and that logic for table formatting. But
# after I understood it I know I would have just made a style sheet and some functions to clean up alot of this code, but it was good 
# practice in learning how to do thigns several ways. 

purchasing_total['Average Price'] = purchasing_total['Average Price'].map('${:,.2f}'.format)
purchasing_total['Total Revenue'] = purchasing_total['Total Revenue'].map('${:,.2f}'.format)

purchasing_total.style.hide_index()

Number of Unique Items,Average Price,Number of Purchases,Total Revenue
179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [46]:
# This is relatively straightforward. Groupby and using nunique gets the unique number per gender. An apply and lambda combo then creates
# a new colume called 'Percentage of Players' and dividing it by 'unique_players' calculated above will yield the percentages.
# The final three lines of code just rename the columns because 'SN' is the value we aggregated over which is the total count and then I 
# formatted to percent and bold first column as the final step. I preferred the set_table_styles method for fonting the columns although
# there are a few ways to do it.

gender_demo = purchase_data.groupby('Gender')['SN'].nunique().reset_index()
gender_demo['Percentage of Players'] = gender_demo['SN'].apply(lambda x: x/unique_players)
gender_demo.rename({'SN': 'Total Count'}, axis=1, inplace=True)
gender_demo['Percentage of Players'] = gender_demo['Percentage of Players'].map('{0:,.2%}'.format)


gender_demo.style.set_table_styles([{'selector': 'td:first-child', 'props': [('font-weight', 'bold')]}]).hide_index()

Gender,Total Count,Percentage of Players
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [47]:
# This analysis just applies the same processes as above. I know the instructions said to run the calculations and then create a summary 
# data frame, but I didn't see the need since I could just use .agg to calculate all my values which seemed cleaner and took fewer lines 
# of code. So I grouped by gender and then ran my aggregations: count of Purchase ID gives number of purchases, summing Price gives TPV, mean of Price
# gives the avg and then I just had to divide TPV by the count of unique players to get average per player. I used apply and lambda as above to get that value
# The last lines just format correctly.

gender_purchase = purchase_data.groupby('Gender').agg({'SN': 'nunique', 'Purchase ID': 'count', 'Price': ['mean','sum']}).reset_index()

# To avoid plagiarism, I used this StackOverflow https://stackoverflow.com/questions/50571793/concise-way-of-flattening-multiindex-columns
# By aggregating over price multiple ways Pandas creates a multindex, so I needed to flatten the index using .to_flat_index()
# to get only one index and then I rename the column names at the end.

gender_purchase.columns = gender_purchase.columns = [f"{x}_{y}" for x, y in gender_purchase.columns.to_flat_index()]

# This line was a bit cumbersome. But it was unclear whether I should divide the average purchase value by purchase per gender or by unique gender. The first would just give
# Average purchase value again so I assumed the second and it matched the homework example. But I recalculated the number of unique players by gender and then deleted the column
# after I used those values to get the Avg total purchase per person. I think I would need to define a lambda function to access values from another data frame, so this was a bit
# of a hack, but not too bad to get the one value. 

gender_purchase['Avg Total Purchase per Person'] = gender_purchase.apply(lambda x: x['Price_sum']/x['SN_nunique'], axis=1)
del gender_purchase['SN_nunique']

# Rename column headers and format to dollar format
gender_purchase.rename({'Gender_': 'Gender', 'Purchase ID_count': 'Purchase Count', 'Price_mean': 'Average Purchase Value', 'Price_sum': 'Total Purchase Value'}, axis=1, inplace=True)
gender_purchase['Average Purchase Value'] = gender_purchase['Average Purchase Value'].map('${:,.2f}'.format)
gender_purchase['Avg Total Purchase per Person'] = gender_purchase['Avg Total Purchase per Person'].map('${:,.2f}'.format)
gender_purchase['Total Purchase Value'] = gender_purchase['Total Purchase Value'].map('${:,.2f}'.format)


# This just hides the index and bolds columns to make it match the format in the readme as above.
gender_purchase.style.set_table_styles([{'selector': 'td:first-child', 'props': [('font-weight', 'bold')]}]).hide_index()
    

Gender,Purchase Count,Average Purchase Value,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [48]:
# Binning using pd.cut is fairly straight forward. I just define my bins, add labels and then create a new column in the df with these values. Then I can groupby this 
# new value and use the same techniques as above to calculate the necessary values.

# These three lines bin the data.
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100] 
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'] 
purchase_data['Age Bracket'] = pd.cut(purchase_data["Age"], bins, labels=group_names)

# I group by nunique Screen Name to get the unique number of players per bin and then after renaming column heads I use an apply/lambda to calculate percents off of my 
# age count and the unique player count calculated above.
age_demo = purchase_data.groupby('Age Bracket')['SN'].nunique().reset_index()
age_demo.rename({'SN': 'Total Count'}, axis=1, inplace=True)
age_demo['Percentage of Players'] = age_demo['Total Count'].apply(lambda row: row/unique_players)

#Format and hide index as per above
age_demo['Percentage of Players'] = age_demo['Percentage of Players'].map('{0:,.2%}'.format)
age_demo.style.set_table_styles([{'selector': 'td:first-child', 'props': [('font-weight', 'bold')]}]).hide_index()

Age Bracket,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [49]:
# The only difference in this section is that binning needs to be done before doing the same process in the gender analysis to derive the same values. In retrospect, I
# realized I should have just defined a function that automates this process. I realized I committed the error of DRY (don't repeat yourself), and that writing a function
# such as def analysis(groupby_category, column_vales): could have been done to save many lines of code in this homework. But since I'm learning, rewriting was helpful to 
# remember the code. 

bins = [0, 9, 14, 19, 24, 29, 34, 39, 100] 
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+'] 

purchase_data['Age Bracket'] = pd.cut(purchase_data["Age"], bins, labels=group_names)
age_purchase = purchase_data.groupby('Age Bracket').agg({'SN': 'nunique', 'Purchase ID': 'count', 'Price': ['mean','sum']}).reset_index()
age_purchase.columns = age_purchase.columns = [f"{x}_{y}" for x, y in age_purchase.columns.to_flat_index()]
age_purchase['Avg Total Purchase per Person'] = age_purchase.apply(lambda x: x['Price_sum']/x['SN_nunique'], axis=1)
del age_purchase['SN_nunique']
age_purchase.rename({'Age Bracket_': 'Age Bracket', 'SN_nunique': 'Total Count', 'Purchase ID_count': 'Purchase Count', 'Price_mean': 'Average Purchase Value', 'Price_sum': 'Total Purchase Value'}, axis=1, inplace=True)
age_purchase['Average Purchase Value'] = age_purchase['Average Purchase Value'].map('${:,.2f}'.format)
age_purchase['Total Purchase Value'] = age_purchase['Total Purchase Value'].map('${:,.2f}'.format)
age_purchase['Avg Total Purchase per Person'] = age_purchase['Avg Total Purchase per Person'].map('${:,.2f}'.format)

age_purchase.style.set_table_styles([{'selector': 'td:first-child', 'props': [('font-weight', 'bold')]}]).hide_index()

Age Bracket,Purchase Count,Average Purchase Value,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [50]:
# Not too much to comment here. I follow the same process I did above where I groupby and then aggregate the values I need. I flatten the index, rename columns and then the unique
# thing here is that I sort by total purchase value. I ran into a troubleshooting issue where when I stringed all of the formatting steps together Jupyter lab kept unsorting
# my data. I just renamed and reset the index of the DF after I had sorted which solved the problem. I could have used inplace=True as well but then would need to do more work to
# get the index to run from 0 to the end of the set. The readme said to get the top 5 spenders so I hide the index and used .head to display the top 5.

top_spenders = purchase_data.groupby('SN').agg({'Purchase ID': 'count', 'Price':['mean', 'sum']}).reset_index()
top_spenders.columns = top_spenders.columns = [f"{x}_{y}" for x, y in top_spenders.columns.to_flat_index()]
top_spenders.rename({'SN_': 'SN', 'Purchase ID_count': 'Purchase Count', 'Price_mean':'Average Purchase Price', 'Price_sum': 'Total Purchase Value'}, axis=1, inplace=True)

#Reset the index based on the sorted values. 
sorted_spenders = top_spenders.sort_values(by='Total Purchase Value', ascending=False).reset_index(drop=True)

sorted_spenders['Average Purchase Price'] = sorted_spenders['Average Purchase Price'].map('${:,.2f}'.format)
sorted_spenders['Total Purchase Value'] = sorted_spenders['Total Purchase Value'].map('${:,.2f}'.format)

sorted_spenders.head(5).style.set_table_styles([{'selector': 'td:first-child', 'props': [('font-weight', 'bold')]}]).hide_index()

SN,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [51]:
# This section is just a rinse and repeat of the Top Spenders, but sorts by item purchase count. There is one caveat. I scanned the data and noticed that the item price varied
# somewhat. For instance, Final Critic is listed at $4.88 and $4.19. The homework example just listed an average of these prices. I assume these items went on sale so the data could
# have been parsed by sale price as well because the 'Item Price' here just gives the average price if there are multiple prices. Also, I listed the top 6 in the header as opposed to 
# 5 in the homework because spots 3-6 were all tied at 9 purchase count each. 

popular_items = purchase_data.groupby(['Item ID', 'Item Name']).agg({"Purchase ID": 'count', 'Price': ['mean', 'sum']}).reset_index()
popular_items.columns = popular_items.columns = [f"{x}_{y}" for x, y in popular_items.columns.to_flat_index()]

item_popularity = popular_items.sort_values(by='Purchase ID_count', ascending=False).reset_index(drop=True)

item_popularity.rename({'Item ID_': 'Item ID', "Item Name_": "Item Name", "Purchase ID_count": "Purchase Count", "Price_mean": "Item Price", 
                        "Price_sum": "Total Purchase Value"}, axis=1, inplace=True)
item_popularity['Item Price'] = item_popularity['Item Price'].map('${:,.2f}'.format)
item_popularity['Total Purchase Value'] = item_popularity['Total Purchase Value'].map('${:,.2f}'.format)


item_popularity.head(6).style.set_table_styles([{'selector': 'td:nth-child(-n + 2)', 'props': [('font-weight', 'bold')]}]).hide_index()


Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [52]:
# This section is even easier since we calculated item popularity already. We simply need to sort by Total Purchase Value instead of Purchase Count and then do the same formatting.

item_profit = popular_items.sort_values(by='Price_sum', ascending=False).reset_index(drop=True)

item_profit.rename({'Item ID_': 'Item Id', "Item Name_": "Item Name", "Purchase ID_count": "Purchase ID", "Price_mean": "Item Price", 
                        "Price_sum": "Total Purchase Value"}, axis=1, inplace=True)
item_profit['Item Price'] = item_profit['Item Price'].map('${:,.2f}'.format)
item_profit['Total Purchase Value'] = item_profit['Total Purchase Value'].map('${:,.2f}'.format)
item_profit.head(5).head(5).style.set_table_styles([{'selector': 'td:nth-child(-n + 2)', 'props': [('font-weight', 'bold')]}]).hide_index()

Item Id,Item Name,Purchase ID,Item Price,Total Purchase Value
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
